# 0.0 Imports

In [1]:
import pandas as pd

from keras.models                    import Sequential
from keras.layers                    import Dense, Dropout, Flatten, Embedding
from keras.preprocessing.text        import Tokenizer
from keras.preprocessing.sequence    import pad_sequences

from sklearn.metrics                 import confusion_matrix
from sklearn.preprocessing           import LabelEncoder
from sklearn.model_selection         import train_test_split

2024-11-10 15:00:13.227441: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 15:00:13.229772: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 15:00:13.288543: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 15:00:13.289808: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 15:00:14.402056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
spam = pd.read_csv( 'spam.csv' )
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
label_enc = LabelEncoder()
y = label_enc.fit_transform( spam['Category'] )
print( y )

[0 0 1 ... 0 0 0]


In [4]:
messages = spam['Message'].values

X_train, X_test, y_train, y_test = train_test_split( messages, y, test_size = 0.3 )

In [5]:
token = Tokenizer( num_words=1000 )
token.fit_on_texts( X_train )

X_train = token.texts_to_sequences( X_train )
X_test = token.texts_to_sequences( X_test )

In [6]:
print( len( X_train ) )

3900


In [7]:
X_train = pad_sequences( X_train, padding='post', maxlen=500 )
X_test = pad_sequences( X_test, padding='post', maxlen=500 )
print( X_train )

[[841 472  39 ...   0   0   0]
 [ 57 142  12 ...   0   0   0]
 [ 85   6 528 ...   0   0   0]
 ...
 [299  39 221 ...   0   0   0]
 [111  40   9 ...   0   0   0]
 [  3  13 633 ...   0   0   0]]


In [8]:
print( len( token.word_index) )

7431


In [9]:
model = Sequential()
model.add( Embedding( input_dim=len( token.word_index ), output_dim=50, input_length=500 ) )
model.add( Flatten() )
model.add( Dense( units=10, activation='relu' ) )
model.add( Dropout( 0.1 ) )
model.add( Dense( units=1, activation='sigmoid' ) )


2024-11-10 15:00:16.402451: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-10 15:00:16.402913: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
model.compile( loss='mean_squared_error', optimizer='adam', metrics=['accuracy'] )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           371550    
                                                                 
 flatten (Flatten)           (None, 25000)             0         
                                                                 
 dense (Dense)               (None, 10)                250010    
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 621571 (2.37 MB)
Trainable params: 621571 (2.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.fit( X_train, y_train, epochs=20, batch_size=10, verbose=True, validation_data=(X_test, y_test) )

Epoch 1/20
390/390 [==============================] - 4s 8ms/step - loss: 0.0733 - accuracy: 0.9128 - val_loss: 0.0417 - val_accuracy: 0.9785
Epoch 2/20
390/390 [==============================] - 3s 7ms/step - loss: 0.0354 - accuracy: 0.9844 - val_loss: 0.0350 - val_accuracy: 0.9785
Epoch 3/20
390/390 [==============================] - 3s 7ms/step - loss: 0.0265 - accuracy: 0.9885 - val_loss: 0.0298 - val_accuracy: 0.9797
Epoch 4/20
390/390 [==============================] - 3s 7ms/step - loss: 0.0207 - accuracy: 0.9915 - val_loss: 0.0265 - val_accuracy: 0.9791
Epoch 5/20
390/390 [==============================] - 3s 7ms/step - loss: 0.0162 - accuracy: 0.9933 - val_loss: 0.0243 - val_accuracy: 0.9809
Epoch 6/20
390/390 [==============================] - 3s 7ms/step - loss: 0.0134 - accuracy: 0.9941 - val_loss: 0.0228 - val_accuracy: 0.9797
Epoch 7/20
390/390 [==============================] - 3s 6ms/step - loss: 0.0109 - accuracy: 0.9946 - val_loss: 0.0210 - val_accuracy: 0.9803
Epoch 

In [12]:
loss, accuracy = model.evaluate( X_test, y_test )
print( 'Loss:', loss )
print( 'Accuracy:', accuracy )

53/53 [==============================] - 0s 4ms/step - loss: 0.0182 - accuracy: 0.9815
Loss: 0.018202733248472214
Accuracy: 0.9814593195915222


In [13]:
new_prev = model.predict( X_test )
print( new_prev )

53/53 [==============================] - 0s 4ms/step
[[1.9732613e-11]
 [1.1372144e-23]
 [4.0002723e-18]
 ...
 [1.8753337e-14]
 [3.6124206e-15]
 [2.4785560e-16]]


In [14]:
prevision = ( new_prev > 0.5 )
print( prevision )

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


In [15]:
cm = confusion_matrix( y_test, prevision )
print( cm )

[[1450    2]
 [  29  191]]
